In [1]:
from IPython.display import Image, FileLink
import pandas as pd
import pandas.rpy.common as com

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(DESeq2)
library(phyloseq)
library(plyr); library(dplyr)
library(ggplot2)
library(doParallel)
library(foreach)
library(reshape)
library(RColorBrewer)
library(gridExtra)
library(stats)
library(data.table)
library(xtable, lib.loc="/home/ashley/R/x86_64-pc-linux-gnu-library/3.1/")
library(stargazer, lib.loc="/home/ashley/R/x86_64-pc-linux-gnu-library/3.1/")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, union, unique,
    unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package:

###This file is from the [sparsity notebook](./Sparsity_make_l2fc_table.ipynb).

In [4]:
%%R
df_all = read.csv("data/l2fc_table.csv")
head(df_all)

  sparsity_threshold Day Treatment     OTU log2FoldChange            p    Rank1
1               0.55  30    13CCSS OTU.783     -0.3370825 9.965741e-01 Bacteria
2               0.55  30    13CCSS  OTU.79      0.7598182 4.889860e-01 Bacteria
3               0.55  30    13CCSS OTU.304      0.4183648 8.089837e-01 Bacteria
4               0.55  30    13CCSS  OTU.50      0.7289694 5.252368e-01 Bacteria
5               0.55  30    13CCSS OTU.382      2.9833574 1.370092e-07 Bacteria
6               0.55  30    13CCSS OTU.465      0.5515994 6.887540e-01 Bacteria
          Rank2      Rank3        Rank4         Rank5       Rank6
1 Acidobacteria Holophagae      Sva0725          <NA>        <NA>
2 Bacteroidetes Cytophagia Cytophagales Cytophagaceae Flexibacter
3 Bacteroidetes Cytophagia Cytophagales Cytophagaceae Flexibacter
4 Bacteroidetes Cytophagia Cytophagales Cytophagaceae Flexibacter
5 Bacteroidetes Cytophagia Cytophagales Cytophagaceae Flexibacter
6 Bacteroidetes Cytophagia Cytophagales Cyto

In [5]:
%%R
nrow (df_all)

[1] 1198


### SS

In [6]:
%%R
FDR = 0.10
df_SS_resp = subset(df_all, (Treatment == "13CCSS")&(padj <= FDR))
nrow (df_SS_resp)

[1] 29


In [7]:
%%R
write.csv(df_SS_resp, file = "data/SS_responders.csv")

In [ ]:
%%R
SS_responders = stargazer(df_SS_resp, type = "latex", summary = FALSE, rownames = FALSE,
                          out="figs/SS_Responders_table.tex")


% Table created by stargazer v.5.1 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Thu, Apr 09, 2015 - 10:25:46 AM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}} ccccccccccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
sparsity\_threshold & Day & Treatment & OTU & log2FoldChange & p & Rank1 & Rank2 & Rank3 & Rank4 & Rank5 & Rank6 & Rank7 & Rank8 & padj \\ 
\hline \\[-1.8ex] 
$0.550$ & $30$ & 13CCSS & OTU.382 & $2.983$ & $0.00000$ & Bacteria & Bacteroidetes & Cytophagia & Cytophagales & Cytophagaceae & Flexibacter & uncultured\_Bacteroidetes\_bacterium &  & $0.00004$ \\ 
$0.550$ & $30$ & 13CCSS & OTU.525 & $1.902$ & $0.002$ & Bacteria & Bacteroidetes & Cytophagia & Cytophagales & Cytophagaceae & Cytophaga & uncultured\_bacterium &  & $0.058$ \\ 
$0.550$ & $30$ & 13CCSS & OTU.285 & $2.517$ & $0.00005$ & Bacteria & Planctomycetes & Planctomycetacia & Planctomycetales & Planctomycetaceae & Pir4\_lineage

In [ ]:
!latex figs/SS_Responders_table.tex >/dev/null
!dvipdfm SS_responder_table.dvi figs/SS_Responders_table.pdf

In [ ]:
FileLink("figs/SS_Responders_table.pdf")

In [ ]:
%%R
df_SS_resp.table = xtable(df_SS_resp[1:10,])
digits(df_SS_resp.table)[c(2,6)] <- 0
print(df_SS_resp.table, floating = FALSE)

In [ ]:
%%R
df_SS_resp_unique = subset(df_SS_resp, !duplicated(OTU))
nrow(df_SS_resp_unique)

In [ ]:
%%R
df_SS_resp_counts = ddply(df_SS_resp, .(Rank2), summarize, NumSubs = length(unique(OTU)))
print (df_SS_resp_counts)

### PS

In [ ]:
%%R
FDR = 0.10
df_PS_resp = subset(df_all, (Treatment == "13CCPS")&(padj <= FDR))
df_PS_resp

In [ ]:
%%R
df_PS_resp_unique = subset(df_PS_resp, !duplicated(OTU))
nrow(df_PS_resp_unique)

In [ ]:
%%R
df_PS_resp_counts = ddply(df_PS_resp, .(Rank2), summarize, NumSubs = length(unique(OTU)))
print (df_PS_resp_counts)